# Pokémon showdown challenge

To start off I needed a way to read out the data files that contain usage stats provided by Smogon*. As a base I used a script made by Kaggle user named 'Fruity Fritz'. I needed to reform a bit of their code to fit my own wants and needs.

*Smogon is a website that provides a lot of information about the Pokémon games. They have a lot of data about the games and the competitive scene. They also have a website where you can play Pokémon games online against other players. This is where the data I am using comes from.

## Pre-processing

In [93]:
import numpy as np
import pandas as pd
import urllib.request as urllib, os
import ssl

In [94]:
# made function to format the same sort of file later
def import_showdown_usage_stats(url, filename): # file url and name for local storage
    # location for datafile
    datafolder = 'data/'
    #download file from url to local storage
    gcontext = ssl.SSLContext() # needed to avoid possible certificate validation error
    downloaded = urllib.urlopen(url, context=gcontext)
    buf = downloaded.read()

    try:
        os.mkdir(datafolder)
    except FileExistsError:
        pass

    with open(datafolder + filename, 'wb') as f:
        f.write(buf)
    # import dataframe
    df = pd.read_csv(datafolder + filename, sep="|",skiprows=5,names=["rank","pokemon","usage","count",],usecols=[1,2,3,4])
    # drop any NaNs we find
    df.dropna(inplace=True)
    # changing data type of the columns
    for k, v in {"rank":int,"pokemon":str, "usage":str, "count":int}.items():
        df[k] = df[k].astype(v)
    # fixing leading/trailing white spaces
    df["pokemon"] = df["pokemon"].str.strip()
    df["usage"] = df["usage"].str.strip()
    # fixing format of string to not use '%'
    df["usage"] = df["usage"].str.rstrip('%')
    # change datatype of usage_%
    df["usage"] = df["usage"].astype(float) / 100
    # get rid of unnecessary columns
    df = df[['pokemon' ,'count' ,'usage']]
    return df

Now we need to use the method above to import and format a usage file from Smogon.

In [95]:
usage_stats = import_showdown_usage_stats(url = "https://www.smogon.com/stats/2023-01/gen9vgc2023series2-0.txt", filename="gen9s2vgc0.txt")
print(usage_stats.shape)
usage_stats

(393, 3)


,pokemon,count,usage
0,Flutter Mane,795029,2.804525e-01
1,Iron Hands,724440,2.555517e-01
2,Amoonguss,705360,2.488211e-01
3,Iron Bundle,663073,2.339040e-01
4,Gholdengo,635767,2.242716e-01
...,...,...,...
388,Smoliv,1,4.000000e-07
389,Bounsweet,1,4.000000e-07
390,Staravia,1,4.000000e-07
391,Nacli,1,4.000000e-07


Then I searched for a complete dataset containing all Pokémon and their various forms. That's where I found a somewhat good [datafile through Kaggle](https://www.kaggle.com/datasets/brdata/complete-pokemon-dataset-gen-iiv). This file will be used to merge onto the Smogon usage datafile and not just the one I imported above. To later validate my findings I plan to use different usage files from different generations, that is why it is important for this datafile to be complete.

I imported it as a csv file and found out the encoding was not the standard UTF-8 encoding, so I just needed to specify the encoding in the read function.

In [96]:
names = ["dexnum","name","type1","type2","total","hp","atk","def","sp.atk","sp.def","spd","variant"]
stats = pd.read_csv("./data/Pokedex_Cleaned.csv",names=names,encoding='windows-1252')
print(stats.shape)
stats.head(10)

(1173, 12)


,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,NaN
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,NaN
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,NaN
3,3,Venusaur,Grass,Poison,625,80,100,123,122,120,80,NaN
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,NaN
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,NaN
6,6,Charizard,Fire,Dragon,634,78,130,111,130,85,100,Mega X
7,6,Charizard,Fire,Flying,634,78,104,78,159,115,100,Mega Y
8,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,NaN
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,NaN


## Data cleaning

There were a lot of NaN values which I needed to fill in a way so that the data is 'complete' and ready to be merged with the usage dataset from Smogon.

In [97]:
stats.type2 = stats.type2.fillna('none') # if the second typing is NaN it means there is no second typing hence the value of none
stats.variant = stats.variant.fillna('') # if the variant is NaN it means there is no variant. replaced with empty string to add variant at the end of the name as done in the usage df
stats.head(10)

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,
3,3,Venusaur,Grass,Poison,625,80,100,123,122,120,80,
4,4,Charmander,Fire,none,309,39,52,43,60,50,65,
5,5,Charmeleon,Fire,none,405,58,64,58,80,65,80,
6,6,Charizard,Fire,Dragon,634,78,130,111,130,85,100,Mega X
7,6,Charizard,Fire,Flying,634,78,104,78,159,115,100,Mega Y
8,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,
9,7,Squirtle,Water,none,314,44,48,65,50,64,43,


After seeing the datafile above I wanted to check the values of both the type columns.

First I wanted to check the type2 column, so I can see if the previous action to fill the NaN values actually worked.

In [98]:
stats.type2.value_counts()

none        540
Flying      118
Psychic      47
Poison       46
Fairy        42
Fighting     42
Ground       41
Steel        38
Dragon       35
Ghost        34
Grass        34
Dark         33
Water        22
Ice          22
Fire         19
Rock         19
Normal       19
Electric     13
Bug           9
Name: type2, dtype: int64

These values are what I expected and also what I want from this column

When inspecting the datafile in another software tool, I found that there were some wierd values in type1 and  wanted to check them out.

In [99]:
stats.type1.value_counts()

Water       147
Normal      130
Grass       102
Bug          99
Psychic      76
Electric     73
Fire         73
Rock         67
Dark         56
Dragon       47
Ground       47
Poison       46
Fighting     46
Ghost        46
Steel        42
Fairy        32
Ice          27
Flying       12
Null          2
Male          1
Female        1
Rockruff      1
Name: type1, dtype: int64

After seeing the wierd values of Rockruff, null, male and female in typings I further looked into them.

In [100]:
stats.loc[(stats['type1'] == "Rockruff") | (stats['type1'] == "Male") | (stats['type1'] == "Female") | (stats['type1'] == "Null")]

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
801,678,Meowstic,Male,Psychic,466,74,48,76,83,81,104,
802,678,Meowstic,Female,Psychic,466,74,48,76,83,81,104,
884,744,Rockruff,Rockruff,Rock,280,45,65,40,30,40,60,Own Tempo
916,772,Type:,Null,Normal,534,95,95,95,95,95,59,
917,772,Type:,Null,none,534,95,95,95,95,95,59,


The null value is from a Pokémon named "type:null", both records are for this same Pokémon. The stats are exactly the same, so I will delete the second mention and just replace the faulty record with a correct one.

The values for male and female are faulty in the sense that they occupy the wrong column, they need to be in variant but are in type1. The Rockruff record is also faulty. the Variant value contains their 'ability' which is another attribute that we do not need in this dataset. I have no idea why it is here.

Because there are only 5 faulty records, I am going to manually replace them.

In [101]:
stats.loc[801] = ['678','Meowstic','Psychic','none','466','74','48','76','83','71','104','Male']
stats.loc[802] = ['678','Meowstic','Psychic','none','466','74','48','76','83','71','104','Female']
stats.loc[884] = ['744','Rockruff','Rock','none','280','45','65','40','30','40','60','']
stats.loc[916] = ['772','Type:Null','Normal','none','534','95','95','95','95','95','59','']
stats.drop(stats.index[917], inplace=True)

stats.type1.value_counts()

Water       147
Normal      131
Grass       102
Bug          99
Psychic      78
Fire         73
Electric     73
Rock         68
Dark         56
Ground       47
Dragon       47
Fighting     46
Poison       46
Ghost        46
Steel        42
Fairy        32
Ice          27
Flying       12
Name: type1, dtype: int64

Later in the process of cleaning the data I found another faulty record. These were the records for the Pokémon named Nidoran. In the games this Pokémon has multiple variants and their gender is displayed next to their name to indicate which variant it is.

In [102]:
stats.loc[(stats['name'] == "Nidoran?")]

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
40,29,Nidoran?,Poison,none,275,55,47,52,40,40,41,
43,32,Nidoran?,Poison,none,273,46,57,40,40,40,50,


In [103]:
stats.loc[40] = ['29','Nidoran','Poison','none','275','55','47','52','40','40','41','Female']
stats.loc[43] = ['32','Nidoran','Poison','none','273','46','57','40','40','40','50','Male']
stats.loc[(stats['name'] == "Nidoran")]

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
40,29,Nidoran,Poison,none,275,55,47,52,40,40,41,Female
43,32,Nidoran,Poison,none,273,46,57,40,40,40,50,Male


As you can see, the weird values are now no longer present in the dataset.

Now we are going to change the notation of the Variant to those that are used by our usage stats dataframe. These notations are the ones that Smogon uses.
When I found this dataset I knew it would take a lot of work to get all the data cleaned. I still chose this one because it was the most complete I could find. Even tough this dataset takes a lot of work to prepare, it only needs data cleaning, other datasets would need to be merged together to create this one and I prefer to clean just one dataset apposed to multiple.

The first thing we are going to do is remove all mega's from the list of Pokémon. This is because mega is a mechanic and not a Pokémon to choose for your team.

In [104]:
stats = stats[stats.variant != 'Mega']
stats = stats[stats.variant != 'Mega X']
stats = stats[stats.variant != 'Mega Y']
stats.variant.value_counts()

                    979
Galarian             16
Alolan               15
Hisuian              13
Therian Form          4
                   ... 
Confined              1
Unbound               1
Baile Style           1
Pa u Style            1
Two-Segment Form      1
Name: variant, Length: 85, dtype: int64

After this I needed to clean the remaining variants. To start with I changed the notation of the regions to the one that Smogon uses.

In [105]:
stats['variant'] = stats['variant'].str.strip()
# notation for region specific variants
stats['variant'] = stats['variant'].replace(['Galarian'],'Galar')
stats['variant'] = stats['variant'].replace(['Alolan'],'Alola')
stats['variant'] = stats['variant'].replace(['Hisuian'],'Hisui')
stats['variant'] = stats['variant'].replace(['Paldean'],'Paldea')

Then I changed the gender notations.

In [106]:
# gender notation
stats['variant'] = stats['variant'].replace(['Female'],'F')
stats['variant'] = stats['variant'].replace(['Male'],'')

Then I removed the suffixes(?).

In [107]:
# removed unneeded endings
stats['variant'] = stats['variant'].str.replace(' Form','')
stats['variant'] = stats['variant'].str.replace(' Style','')
stats['variant'] = stats['variant'].str.replace(' Super','')
stats['variant'] = stats['variant'].str.replace(' Size','')
stats['variant'] = stats['variant'].str.replace(' Plumage','')

After that I removed records of variant that do not change the stats of the Pokémon.

In [108]:
# variants that dont matter in stats or are not available in Pokemon showdown
stats = stats[stats.variant != 'Blue-Striped']
stats = stats[stats.variant != 'Red-Striped']
stats = stats[stats.variant != 'Partner']
stats = stats[stats.variant != 'Altered']
stats = stats[stats.variant != 'Eternamax']

There were still some Pokémon with multiple variants that did not have a base variant or empty variant. Those still needed fixing.

In [109]:
# base forms that have a variant fixes
stats['variant'] = stats['variant'].replace('White-Striped','')
stats['variant'] = stats['variant'].replace('Hero of Many Battles','')
stats['variant'] = stats['variant'].replace('Incarnate','')
stats['variant'] = stats['variant'].replace('Ordinary','')
stats['variant'] = stats['variant'].replace('Aria','')
stats['variant'] = stats['variant'].replace('Shield','')
stats['variant'] = stats['variant'].replace('Average','')
stats['variant'] = stats['variant'].replace('50%','')
stats['variant'] = stats['variant'].replace('Confined','')
stats['variant'] = stats['variant'].replace('Baile','')
stats['variant'] = stats['variant'].replace('Midday','')
stats['variant'] = stats['variant'].replace('Solo','')
stats['variant'] = stats['variant'].replace('Core','')
stats['variant'] = stats['variant'].replace('Single Strike','')
stats['variant'] = stats['variant'].replace('Full Belly Mode','')
stats['variant'] = stats['variant'].replace('Family of Three','')
stats['variant'] = stats['variant'].replace('Family of Four','')
stats['variant'] = stats['variant'].replace('Amped','')
stats['variant'] = stats['variant'].replace('Two-Segment','')
stats['variant'] = stats['variant'].replace('Green','')
stats['variant'] = stats['variant'].replace('Zero','')

Now I needed to manually fix the variants that still needed fixes.

In [110]:
# unique variant notation fixes
stats['variant'] = stats['variant'].replace('Crowned Sword','Crowned')
stats['variant'] = stats['variant'].replace('Crowned Shield','Crowned')
stats['variant'] = stats['variant'].replace('Ice Rider','Ice')
stats['variant'] = stats['variant'].replace('Shadow Rider','Shadow')
stats['variant'] = stats['variant'].replace('Galarian Zen','Galar-Zen')
stats['variant'] = stats['variant'].replace('Pa u',"Pa'u")
stats['variant'] = stats['variant'].replace('Rapid Strike','Rapid-Strike')
stats['variant'] = stats['variant'].replace('Hangry Mode','Hangry')
stats['variant'] = stats['variant'].replace('Low Key','Low-Key')
stats['variant'] = stats['variant'].replace('Galarian Standard','Galar')

stats.variant.value_counts()

                 1009
Galar              17
Alola              15
Hisui              13
Therian             4
F                   4
Origin              3
Super               2
Small               2
Primal              2
White               2
Sandy Cloak         2
Trash Cloak         2
Large               2
Crowned             2
Plant Cloak         2
Dusk                1
Midnight            1
School              1
Meteor              1
Dawn Wings          1
Hangry              1
Dusk Mane           1
Ultra               1
Low-Key             1
Pom-Pom             1
Rapid-Strike        1
Ice                 1
Shadow              1
Blue                1
Yellow              1
Hero                1
Sensu               1
Blade               1
Pa'u                1
Mow                 1
Paldea              1
Rainy               1
Snowy               1
Sunny               1
Fan                 1
Frost               1
Heat                1
Wash                1
Unbound             1
Galar-Zen 

After this there were still 6 variants that were not yet fully clean. These were the Pokémon: Burmy and Wormadam. These two Pokémon have the same shared 3 Variants, but those variants only matter for Wormadam and not for Burmy. This means I will need to manually these values. the Variants in question are the Plant Cloak, Sandy Cloak and Trash Cloak variants. The Plant cloak is the easiest to handle because this is the 'default' form according to the [Smogon Pokédex](https://www.smogon.com/dex/sv/pokemon/), this means that an empty('') value would be fitting for both Pokémon. The naming of cloak in the variant needs to be removed for the same reason.
 The only thing left to do was to remove the remaining Burmy, the way I found easiest was to locate them and manually remove the unneeded records.

In [111]:
stats['variant'] = stats['variant'].replace('Plant Cloak','')
stats['variant'] = stats['variant'].str.replace(' Cloak','')
stats.loc[(stats['name'] == "Burmy") | (stats['name'] == "Wormadam")]

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
493,412,Burmy,Bug,none,224,40,29,45,29,45,36,
494,412,Burmy,Bug,none,224,40,29,45,29,45,36,Sandy
495,412,Burmy,Bug,none,224,40,29,45,29,45,36,Trash
499,413,Wormadam,Bug,Grass,424,60,59,85,79,105,36,
500,413,Wormadam,Bug,Ground,424,60,79,105,59,85,36,Sandy
501,413,Wormadam,Bug,Steel,424,60,69,95,69,95,36,Trash


In [112]:
stats.drop(stats.index[494], inplace=True)
stats.drop(stats.index[495], inplace=True)
stats.loc[(stats['name'] == "Burmy") | (stats['name'] == "Wormadam")]

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
493,412,Burmy,Bug,none,224,40,29,45,29,45,36,
494,412,Burmy,Bug,none,224,40,29,45,29,45,36,Sandy
495,412,Burmy,Bug,none,224,40,29,45,29,45,36,Trash
499,413,Wormadam,Bug,Grass,424,60,59,85,79,105,36,
500,413,Wormadam,Bug,Ground,424,60,79,105,59,85,36,Sandy
501,413,Wormadam,Bug,Steel,424,60,69,95,69,95,36,Trash


This did not work and I started thinking again, I asked myself if I really did need to transform this data. I didn't really need to strip out the Burmy forms. This because I want to join this dataset on the one provided by Smogon. This would mean it will only select the one with the correct variant index. In this case that index wil always by none because Smogon does not recognize the other Burmy variants. This makes the step above unnecessary.

After a quick inspection of the dataset I found out that there are duplicate rows.

In [113]:
stats.duplicated().__contains__(True)

True

In [114]:
stats.drop_duplicates()

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,
3,3,Venusaur,Grass,Poison,625,80,100,123,122,120,80,
4,4,Charmander,Fire,none,309,39,52,43,60,50,65,
...,...,...,...,...,...,...,...,...,...,...,...,...
1168,1004,Chi-Yu,Dark,Fire,570,55,80,80,135,120,100,
1169,1005,Roaring Moon,Dragon,Dark,590,105,139,71,55,101,119,
1170,1006,Iron Valiant,Fairy,Fighting,590,74,130,90,120,60,116,
1171,1007,Koraidon,Fighting,Dragon,670,100,135,115,85,100,135,


I also found out that the dataset is incomplete. I will add the missing Pokémon manually here, this section will be updated if I find more missing Pokémon later.

In [115]:
stats.loc[len(stats)] = [128,"Tauros","Fighting","Water", 490, 75, 110, 105, 30, 70, 100,"Paldea-Aqua"]
stats.loc[len(stats) + 1] = [128,"Tauros","Fighting","Fire", 490, 75, 110, 105, 30, 70, 100,"Paldea-Blaze"]
stats.loc[len(stats) + 2] = [128,"Tauros","Fighting","none", 490, 75, 110, 105, 30, 70, 100,"Paldea-Combat"]
stats.loc[stats['dexnum'] == 128]

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant
164,128,Tauros,Normal,none,490,75,100,95,40,70,110,
1118,128,Tauros,Fighting,Water,490,75,110,105,30,70,100,Paldea-Aqua
1119,128,Tauros,Fighting,Fire,490,75,110,105,30,70,100,Paldea-Blaze
1120,128,Tauros,Fighting,none,490,75,110,105,30,70,100,Paldea-Combat


Now I want to add a column containing the generation of the Pokémon game that the Pokémon was first introduced, this is because I think it would have significance to figure out reasons why certain Pokémon are used the most.

In [116]:
stats['dexnum'] = stats['dexnum'].astype(int)
gen = []
for index, row in stats.iterrows():
    if row['dexnum'] <= 151:
        gen.append(1)
    elif row['dexnum'] > 151 and row['dexnum'] <= 251:
        gen.append(2)
    elif row['dexnum'] > 251 and row['dexnum'] <= 386:
        gen.append(3)
    elif row['dexnum'] > 386 and row['dexnum'] <= 493:
        gen.append(4)
    elif row['dexnum'] > 493 and row['dexnum'] <= 649:
        gen.append(5)
    elif row['dexnum'] > 649 and row['dexnum'] <= 721:
        gen.append(6)
    elif row['dexnum'] > 721 and row['dexnum'] <= 809:
        gen.append(7)
    elif row['dexnum'] > 809 and row['dexnum'] <= 905:
        gen.append(8)
    elif row['dexnum'] > 905:
        gen.append(9)

stats['generation'] = gen
stats

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant,generation
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,,1
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,,1
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,,1
3,3,Venusaur,Grass,Poison,625,80,100,123,122,120,80,,1
4,4,Charmander,Fire,none,309,39,52,43,60,50,65,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,1004,Chi-Yu,Dark,Fire,570,55,80,80,135,120,100,,9
1169,1005,Roaring Moon,Dragon,Dark,590,105,139,71,55,101,119,,9
1170,1006,Iron Valiant,Fairy,Fighting,590,74,130,90,120,60,116,,9
1171,1007,Koraidon,Fighting,Dragon,670,100,135,115,85,100,135,,9


After inspecting the data for generation I found out that the column was successfully added with the correct data.

I think it is now time to combine the Pokémon and variant column to ready this dataframe for a merge

In [117]:
pokemon_names = []
for index, row in stats.iterrows():
    if len(row.variant) >= 1:
        pokemon_names.append(row['name'] + "-" + row['variant'])
    else:
        pokemon_names.append(row['name'])
stats['pokemon'] = pokemon_names
stats

,dexnum,name,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,variant,generation,pokemon
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,,1,Bulbasaur
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,,1,Ivysaur
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,,1,Venusaur
3,3,Venusaur,Grass,Poison,625,80,100,123,122,120,80,,1,Venusaur
4,4,Charmander,Fire,none,309,39,52,43,60,50,65,,1,Charmander
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,1004,Chi-Yu,Dark,Fire,570,55,80,80,135,120,100,,9,Chi-Yu
1169,1005,Roaring Moon,Dragon,Dark,590,105,139,71,55,101,119,,9,Roaring Moon
1170,1006,Iron Valiant,Fairy,Fighting,590,74,130,90,120,60,116,,9,Iron Valiant
1171,1007,Koraidon,Fighting,Dragon,670,100,135,115,85,100,135,,9,Koraidon


Now I wanted to drop the columns that I previously merged togother, this is because I don't need them anymore.

In [118]:
stats = stats.drop(columns=['name', 'variant'])
stats

,dexnum,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,generation,pokemon
0,1,Grass,Poison,318,45,49,49,65,65,45,1,Bulbasaur
1,2,Grass,Poison,405,60,62,63,80,80,60,1,Ivysaur
2,3,Grass,Poison,525,80,82,83,100,100,80,1,Venusaur
3,3,Grass,Poison,625,80,100,123,122,120,80,1,Venusaur
4,4,Fire,none,309,39,52,43,60,50,65,1,Charmander
...,...,...,...,...,...,...,...,...,...,...,...,...
1168,1004,Dark,Fire,570,55,80,80,135,120,100,9,Chi-Yu
1169,1005,Dragon,Dark,590,105,139,71,55,101,119,9,Roaring Moon
1170,1006,Fairy,Fighting,590,74,130,90,120,60,116,9,Iron Valiant
1171,1007,Fighting,Dragon,670,100,135,115,85,100,135,9,Koraidon


In [119]:
df_gen9 = pd.merge(usage_stats, stats, on='pokemon', how='inner')
print(df_gen9.shape)
df_gen9

(385, 14)


,pokemon,count,usage,dexnum,type1,type2,total,hp,atk,def,sp.atk,sp.def,spd,generation
0,Flutter Mane,795029,2.804525e-01,987,Ghost,Fairy,570,55,55,55,135,135,135,9
1,Iron Hands,724440,2.555517e-01,992,Fighting,Electric,570,154,140,108,50,68,50,9
2,Amoonguss,705360,2.488211e-01,591,Grass,Poison,464,114,85,70,85,80,30,5
3,Iron Bundle,663073,2.339040e-01,991,Ice,Water,570,56,80,114,124,60,136,9
4,Gholdengo,635767,2.242716e-01,1000,Steel,Ghost,550,87,60,95,133,91,84,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,Smoliv,1,4.000000e-07,928,Grass,Normal,260,41,35,45,58,51,30,9
381,Bounsweet,1,4.000000e-07,761,Grass,none,210,42,30,38,30,38,32,7
382,Staravia,1,4.000000e-07,397,Normal,Flying,340,55,75,50,40,40,80,4
383,Nacli,1,4.000000e-07,932,Rock,none,280,55,55,75,35,35,25,9


## Data Analysis

In [124]:
from matplotlib import pyplot as plt
from pandas.plotting import parallel_coordinates
import seaborn as sns
%matplotlib inline

In [129]:
plot_data = df_gen9[['count', 'hp', 'atk', 'def', 'sp.atk', 'sp.def', 'spd']]

sns.pairplot(plot_data, hue="count")

ValueError: No variables found for grid columns.